In [1]:
import csv
from tabulate import tabulate

# configure below to print tables for Jupyter notebook or for Latex
tablestyle = "fancy_grid"
# tablestyle = "latex"

def nano_to_milli(x):
    return round(x * 10**(-6),1)

error_margin = 0.03

In [2]:
header = ['alg','opt','dim','len','max','neg','time','unit','dev']
table_vals1 = []

with open('strat01a/data.csv', newline='') as csvfile:
    results_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    ctr = 0
    for row in results_reader:
        #print(', '.join(row))
        if ctr > 0:
            table_vals1.append(row)
        ctr += 1

#print(tabulate(row, headers=header, tablefmt=tablestyle))
#table_vals1

In [3]:
header = ['alg','opt','dim','len','max','neg','time','unit','dev']
table_vals2 = []

with open('strat01b/data.csv', newline='') as csvfile:
    results_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    ctr = 0
    for row in results_reader:
        #print(', '.join(row))
        if ctr > 0:
            table_vals2.append(row)
        ctr += 1
        
#print(tabulate(row, headers=header, tablefmt=tablestyle))
#table_vals2

In [4]:
header = ['alg','opt','dim','len','max','neg','time','unit','dev']
table_vals3 = []

with open('strat02/data.csv', newline='') as csvfile:
    results_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    ctr = 0
    for row in results_reader:
        #print(', '.join(row))
        if ctr > 0:
            table_vals3.append(row)
        ctr += 1
        
#print(tabulate(table_vals3, headers=header, tablefmt=tablestyle))
#table_vals3

In [5]:
def collect_pos_vals(header_val, table_vals):
    lis_vals = []
    for row in table_vals:
        tmp = row[header.index(header_val)]
        if not tmp in lis_vals:
            lis_vals.append(tmp)
    return lis_vals
        
algs1 = collect_pos_vals('alg', table_vals1)
opts1 = collect_pos_vals('opt', table_vals1)
dims1 = collect_pos_vals('dim', table_vals1)
lens1 = collect_pos_vals('len', table_vals1)
maxes1 = collect_pos_vals('max', table_vals1)
neg_degrees1 = collect_pos_vals('neg', table_vals1)
#print("Data set 1a", dims1, lens1, maxes1)

algs2 = collect_pos_vals('alg', table_vals2)
opts2 = collect_pos_vals('opt', table_vals2)
dims2 = collect_pos_vals('dim', table_vals2)
lens2 = collect_pos_vals('len', table_vals2)
maxes2 = collect_pos_vals('max', table_vals2)
neg_degrees2 = collect_pos_vals('neg', table_vals2)
#print("Data set 1b", dims2, lens2, maxes2)

algs3 = collect_pos_vals('alg', table_vals3)
opts3 = collect_pos_vals('opt', table_vals3)
dims3 = collect_pos_vals('dim', table_vals3)
lens3 = collect_pos_vals('len', table_vals3)
maxes3 = collect_pos_vals('max', table_vals3)
neg_degrees3 = collect_pos_vals('neg', table_vals3)
#print("Data set 2", dims3, lens3, maxes3)

In [6]:
def collect_corresponding_val(table_vals, opt_val, algo, dim_val, max_val, neg_degree, len_val):
    for row in table_vals:
        test0 = row[header.index('opt')] == opt_val
        test1 = row[header.index('alg')] == algo
        test2 = row[header.index('dim')] == dim_val
        test3 = row[header.index('max')] == max_val
        test4 = row[header.index('neg')] == neg_degree
        test5 = row[header.index('len')] == len_val
        if test0 and test1 and test2 and test3 and test4 and test5:
            return int(row[header.index('time')])
    print('not found')
    return 10**10

In [7]:
relevant_dups = [dims3[0], dims3[3], dims3[4]]

### Summary of the results to extract from these data sets:

We set the policy length to 60 (to unify among measurements taken) 

For each optimization (negations degrees 0 (pos) and 1 (neg)), we want to obtain:
- max slowdown factor compared to baseline (opt1) for keygen and encrypt (among all duplication cases) 
- best decrypt performance per dup case
- max speedup factor compared to baseline (opt1) for decrypt per dup case (regular, auth, auth_attr, auth_lbl)

In [8]:
def extract_max_slowdown_keygen_encrypt(algo, opt_val, n_d):
    
    # first we check the 'regular' benchmark
    baseline = collect_corresponding_val(table_vals1, opts1[1], algo, dims1[0], maxes1[0], n_d, lens1[59])
    cand_opt_timing = collect_corresponding_val(table_vals1, opt_val, algo, dims1[0], maxes1[0], n_d, lens1[59])
    max_slowdown = round(cand_opt_timing/baseline,3) 
    
    # now check among various duplication cases
    for dim_val in dims3:
        for len_val in lens3:
            cand_opt_timing = collect_corresponding_val(table_vals3, opt_val, algo, dim_val, maxes3[0], n_d, len_val)
            sf = round(cand_opt_timing/baseline,3) 
            if sf > max_slowdown:
                max_slowdown = sf
    
    return max_slowdown

In [9]:
# 1 is keygen and 2 is encrypt
#print(extract_max_slowdown_keygen_encrypt(algs1[2], opts1[8], neg_degrees1[1]))

In [10]:
def extract_min_costs_decrypt_one_dim_val(opt_val, n_d, dim_val):
    
    if dim_val == 'regular':
        cand_opt_timing = collect_corresponding_val(table_vals1, opt_val, 'decrypt', dims1[0], maxes1[0], n_d, lens1[59])
        min_costs = cand_opt_timing
        
    else:
        # here we have to sample over lens3, i.e., [1,2,3...,30,60]
        min_costs = 10**10
        for len_val in lens3:
            cand_opt_timing = collect_corresponding_val(table_vals3, opt_val, 'decrypt', dim_val, maxes3[0], n_d, len_val)
            if cand_opt_timing < min_costs:
                min_costs = cand_opt_timing
    
    return min_costs

In [11]:
def extract_max_speedup_decrypt_one_dim_val(opt_val, n_d, dim_val):
    
    if dim_val == 'regular':
        baseline = collect_corresponding_val(table_vals1, opts1[1], 'decrypt', dims1[0], maxes1[0], n_d, lens1[59])
        cand_opt_timing = collect_corresponding_val(table_vals1, opt_val, 'decrypt', dims1[0], maxes1[0], n_d, lens1[59])
        max_speedup = round(baseline/cand_opt_timing,3) 
        
    else:
        # here we have to sample over lens3, i.e., [1,2,3...,30,60]
        max_speedup = 0
        for len_val in lens3:
            baseline = collect_corresponding_val(table_vals3, opts1[1], 'decrypt', dim_val, maxes3[0], n_d, len_val)
            cand_opt_timing = collect_corresponding_val(table_vals3, opt_val, 'decrypt', dim_val, maxes3[0], n_d, len_val)
            speedup = round(baseline/cand_opt_timing,3) 
            if speedup > max_speedup:
                max_speedup = speedup
    
    return max_speedup

In [12]:
def extract_min_speedup_decrypt_one_dim_val(opt_val, n_d, dim_val):
    
    if dim_val == 'regular':
        baseline = collect_corresponding_val(table_vals1, opts1[1], 'decrypt', dims1[0], maxes1[0], n_d, lens1[59])
        cand_opt_timing = collect_corresponding_val(table_vals1, opt_val, 'decrypt', dims1[0], maxes1[0], n_d, lens1[59])
        min_speedup = round(baseline/cand_opt_timing,3) 
        
    else:
        # here we have to sample over lens3, i.e., [1,2,3...,30,60]
        min_speedup = 10**10
        for len_val in lens3:
            #baseline = collect_corresponding_val(table_vals3, opts1[1], 'decrypt', dim_val, maxes3[0], n_d, len_val)
            baseline = collect_corresponding_val(table_vals1, opts1[1], 'decrypt', dims1[0], maxes1[0], n_d, lens1[59])
            cand_opt_timing = collect_corresponding_val(table_vals3, opt_val, 'decrypt', dim_val, maxes3[0], n_d, len_val)
            speedup = round(baseline/cand_opt_timing,3) 
            if speedup < min_speedup:
                min_speedup = speedup
    
    return min_speedup

In [13]:
def create_summary_table():
    nd0 = neg_degrees1[0]
    nd1 = neg_degrees1[1]
    opts = opts1[1:]
    
    header1 = ['', '', 'KG', 'E', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']
    header2 = ['', '', '', '', 'reg', 'reg', 'a', 'a', 'a_l', 'a_l', 'a_a', 'a_a']
    header3 = ['', '', 'MSlF', 'MSlF', 'min', 'MSpF', 'min', 'MSpF', 'min', 'MSpF', 'min', 'MSpF']
    
    # to select positive row for opts[i], use
    # summary_table[2+2*opts.index(opts[i])]
    # to select negative row for opts[i], use
    # summary_table[2+2*opts.index(opts[i]) + 1]
    
    summary_table = [header2, header3]
    for opt_val in opts:
        summary_table.append([opt_val, 'pos'])
        summary_table.append([opt_val, 'neg'])
        
    # first add the slowdown factors for keygen and encrypt
    for opt_val in opts:
        for algo in [algs1[1], algs1[2]]:
            summary_table[2+2*opts.index(opt_val)].append(extract_max_slowdown_keygen_encrypt(algo, opt_val, nd0))
            summary_table[2+2*opts.index(opt_val) + 1].append(extract_max_slowdown_keygen_encrypt(algo, opt_val, nd1))
    
    # now add min costs and speedup for decrypt
    for opt_val in opts:
        for dim_val in ['regular'] + relevant_dups:
            summary_table[2+2*opts.index(opt_val)].append(nano_to_milli(extract_min_costs_decrypt_one_dim_val(opt_val, nd0, dim_val)))
            summary_table[2+2*opts.index(opt_val) + 1].append(nano_to_milli(extract_min_costs_decrypt_one_dim_val(opt_val, nd1, dim_val)))
            minimax1 = str(extract_min_speedup_decrypt_one_dim_val(opt_val, nd0, dim_val)) + '-' + str(extract_max_speedup_decrypt_one_dim_val(opt_val, nd0, dim_val))
            summary_table[2+2*opts.index(opt_val)].append(minimax1)
            minimax2 = str(extract_min_speedup_decrypt_one_dim_val(opt_val, nd1, dim_val)) + '-' + str(extract_max_speedup_decrypt_one_dim_val(opt_val, nd1, dim_val))
            summary_table[2+2*opts.index(opt_val) + 1].append(minimax2)
    
    print(tabulate(summary_table, headers=header1, tablefmt=tablestyle))


In [14]:
create_summary_table()

╒══════╤═════╤═══════╤═══════╤═══════╤═════════════╤═══════╤══════════════╤═══════╤═════════════╤═══════╤══════════════╕
│      │     │ KG    │ E     │ D     │ D           │ D     │ D            │ D     │ D           │ D     │ D            │
╞══════╪═════╪═══════╪═══════╪═══════╪═════════════╪═══════╪══════════════╪═══════╪═════════════╪═══════╪══════════════╡
│      │     │       │       │ reg   │ reg         │ a     │ a            │ a_l   │ a_l         │ a_a   │ a_a          │
├──────┼─────┼───────┼───────┼───────┼─────────────┼───────┼──────────────┼───────┼─────────────┼───────┼──────────────┤
│      │     │ MSlF  │ MSlF  │ min   │ MSpF        │ min   │ MSpF         │ min   │ MSpF        │ min   │ MSpF         │
├──────┼─────┼───────┼───────┼───────┼─────────────┼───────┼──────────────┼───────┼─────────────┼───────┼──────────────┤
│ opt1 │ pos │ 1.003 │ 1.006 │ 193.2 │ 1.0-1.0     │ 193.5 │ 0.998-1.0    │ 193.4 │ 0.998-1.0   │ 100.3 │ 0.999-1.0    │
├──────┼─────┼───────┼───────┼──